# STAC (Simultaneous Tracking and Calibration) 

In this notebook, we will explore how STAC can register a biomechanical model of rats to 3D pose estimates obtained from DANNCE markerless pose estimation. This algorithm is designed to accurately track the movements of rats in three-dimensional space, which can be incredibly useful in a variety of research areas, including neuroscience, biophysics, and biomechanics.

STAC is an algorithm that can simultaneously calibrate a biomechanical model to keypoints and  by learning from data. Throughout this notebook, we will walk through the steps of implementing STAC, including configuration files, model calibration, and tracking. We will also discuss the theory behind the algorithm and how it works to accurately register a biomechanical model to 3D pose estimates.

In [25]:
from stac import compute_stac, view_stac, viz

In [26]:
data_path = "data.mat"
param_path = "params.yaml"
offset_path = "offset.p"
results_path = "results.p"
start_frame = 0
end_frame_fitting = start_frame + 10 # 2 seconds of data at 50 fps
end_frame_transform = start_frame + 500 # 10 seconds of data at 50 fps

# Calibration



The calibration step is a crucial part of the STAC algorithm. It involves using a set of poses to calibrate the biomechanical model and determine the relationship between the model and the 3D pose estimates. This step ensures that the model accurately represents the movement of the rat. In this notebook, we will demonstrate how to `fit` the model using a short set of poses. While the original implementation of STAC called for joint optimization of tracking and calibration across the entire dataset, in practice we find that doing so on a small sample of data is sufficient. 

In [27]:
st = compute_stac.STAC(
    data_path,
    param_path,
    start_frame=start_frame,
    end_frame=end_frame_fitting)
st.fit().save(save_path=offset_path)

# Tracking

Now that we have calibrated the biomechanical model, we can use `transform` to track the rat's movements in 3D space. The algorithm optimizes a cost function that balances the error between the predicted and observed keypoints, while also considering the dynamics of the rat's movement. In this notebook, we will demonstrate how to set up the tracking configuration, load the data, and run the optimization. We will also discuss how to fine-tune the tracking parameters to achieve the best possible results.

In [28]:
st.end_frame = end_frame_transform
data = st.transform(offset_path)
st.save(save_path=results_path)
